# Introduccion al Reconocimiento del Habla - Tutorial ESPnet

Ref:
- https://colab.research.google.com/drive/1L85G7jdhsI1QKs2o0qCGEbhm5X4QV2zN?usp=sharing

Autor Original: Shinji Watanabe
    
ESPnet is un tolkit para procesamiento de voz end-to-end (fin-a-fin), inicialmente aplicado al reconocimiento del habla fin-a-fin
y la sintetizacion de voz (texto-a-voz) fin-a-fin. En la actualidad este toolkit ha sido extendido a otros aplicaciones del procesamiento
del habla. ESPnet usa PyTorch como motor principal para el aprendizaje profundo.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Ejemplo del reconocimiento

- ESPnet cuenta con varias aplicaciones dedicadas al procesamiento de voz y tiene modelos preentrenados
- Podemos verificar los modelos en [espnet_model_zoo](https://github.com/espnet/espnet_model_zoo)


In [ ]:
!pip install -q espnet_model_zoo

Ahora escogemos el idioma de nuestro reconocedor

In [ ]:
lang = 'en'
fs = 16000
tag = 'Shinji Watanabe/spgispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_unnorm_bpe5000_valid.acc.ave'

### Inicializacion del Modelo

In [ ]:
import time
import torch
import string
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text


d = ModelDownloader("/content/drive/MyDrive/Data_NB/pretrained")
# It may takes a while to download and build models
speech2text = Speech2Text(
    **d.download_and_unpack(tag),
    device="cuda",
    minlenratio=0.0,
    maxlenratio=0.0,
    ctc_weight=0.3,
    beam_size=10,
    batch_size=0,
    nbest=1
)

def text_normalizer(text):
    text = text.upper()
    return text.translate(str.maketrans('', '', string.punctuation))

### Reconocimiento de nuestras grabaciones

In [ ]:
!pip install git+git://github.com/ricardodeazambuja/colab_utils.git

In [ ]:
from colab_utils import getAudio
import numpy as np
from matplotlib import pyplot as plt
from librosa import resample

In [ ]:
speech, fs = getAudio()
dtype = speech.dtype
speech = speech.astype(np.float32) / (np.iinfo(dtype).max + 1)
speech = resample(speech, fs, 16000)
plt.plot(speech)

In [ ]:
hyp = 'Today I will share information about deep learning'
nbests = speech2text(speech)

text, *_ = nbests[0]
print(f"ASR hypothesis: {text_normalizer(text)}")

## Instalacion completa

Para el proceso de Preprocesamiento, entrenamiento, inferencia, evaluacion, necesitamos contar con una instalacion completa  
del toolkit.
Recuerden que este toolkit solo funciona en **Linux** (Ubuntu de preferencia.)
Muchas de las herramientas no estan disponibles para **Windows**. Asi que no se recomienda su uso completo.


In [ ]:
# Descargar ESPnet
%cd /content/drive/MyDrive/Data_NB/
# OS setup
!cat /etc/os-release
!apt-get install -qq bc tree sox
!chmod a+x espnet/utils/*

In [ ]:
# espnet setup
!pip install chainer==6.0.0
# !git clone --depth 5 https://github.com/espnet/espnet
# !cd espnet; pip install -q -e .

# download pre-compiled warp-ctc and kaldi tools
!espnet/utils/download_from_google_drive.sh \
    "https://drive.google.com/open?id=13Y4tSygc8WtqzvAVGK_vRV9GlV7TRC0w" espnet/tools tar.gz > /dev/null
!cd espnet/tools && bash installers/install_sph2pipe.sh

# make dummy activate
!rm espnet/tools/activate_python.sh 
!touch espnet/tools/activate_python.sh
!echo "setup done."
%rm -rf espnet/tools/kaldi
!git clone https://github.com/kaldi-asr/kaldi

In [ ]:
# Verificamos la correcta instalacion
%cd /content/espnet/tools
!. ./activate_python.sh; python3 check_install.py

## Empleando una receta

ESPnet cuenta con un numero de recetas (73 en total hasta el dia 16 de Sept. 2021).

Para los siguientes pasos, utilizaremos la guia general descrita en: https://espnet.github.io/espnet/espnet2_tutorial.html#recipes-using-espnet2

### CMU AN4

Para este club emplearemos como muestra la receta de la base de datos AN4.  
Es una tarea de pequeña escala principalmente destinada a evaluaciones de la plataforma.


In [ ]:
# Nos movemos al directorio
%cd /content/drive/MyDrive/Data_NB/espnet/egs2/an4/asr1
!ls

La ejecucion de la receta se realiza a traves de `run.sh` que llama a `asr.sh`. Este archivo completa el experimento del reconocimiento del habla, incluyendo preparacion de los datos, entrenamiento, inferencia y puntuacion. En total se cuenta con 15 diferentes etapas.

In [ ]:
!cat run.sh

In [ ]:
!chmod a+x local/*.sh
!chmod a+x utils/*.sh
!chmod a+x utils/*.pl
!chmod a+x step/*.sh

### Etapa 1: Preparacion

In [ ]:
!./run.sh --stage 1 --stop_stage 1

In [ ]:
!ls data

### Etapa 2: Perturbacion de la velociadad

Para este ejercicio, no emplearemos la perturbacion. Usualmente empleada para generar nuevos datos
(incremento de la base de datos, o data augmentation).

### Etapa 3: Formato de wav.scp: data/ -> dump/raw
Convertimos los datos en un formato especifico (para este caso: flac) que hace un uso eficiente de la memoria.


In [ ]:
!./run.sh --stage 3 --stop_stage 3 --nj 4

### Etapa 4: Eliminar archivos de corta o larga duracion
Archivos de muy corta o larga duracion pueden ser dañinos al entrenamiendo. Estos archivos deben ser retirados de la lista.

In [ ]:
!./run.sh --stage 4 --stop_stage 4

### Etapa 5: Generacion de una lista de tokens (caracteres) usando BPE.
Esta es una etapa importante para el procesamiento de texto. Hacemos un diccionario basado en carateres en inglés para este ejercicio.  
Para esto, empleamos la herramienta `sentecepiece` desarrollada por Google.

In [ ]:
!./run.sh --stage 5 --stop_stage 5

Revisamos el diccionario, en el cual podremos encontrar caracteres especiales:

In [ ]:
!cat data/token_list/bpe_unigram30/tokens.txt

### Modelamiento del Lenguaje (De momento lo saltamos - Analizaremos en el dia 3-4)
**Etapas 6 ~ 9: Relacionadas al modelamiento de lenguaje**
    

## Reconocimiento del Habla (ASR) End-to-End

### Etapa 10: Recoleccion de estadisticas ASR

En esta etapa, estimamos la media y la varianza de nuestros datos para normalizarlos.  
Tambien recogemos informacion acerca de las longitudes de entrada y salida para una
generacion eficiente de los conjuntos de entrenamiento (mini-batch)

In [ ]:
!./run.sh --stage 10 --stop_stage 10

### Etapa 11: Entrenamiento del modelo
Esta es la ejecucion principal del entrenamiento

Monitorear los siguientes archivos:
- Archivo de Registro: /content/espnet/egs2/an4/asr1/exp/asr_train_raw_bpe30/train.log
- Error: /content/espnet/egs2/an4/asr1/exp/asr_train_raw_bpe30/images/loss.png
- Exactitud: /content/espnet/egs2/an4/asr1/exp/asr_train_raw_bpe30/images/acc.png


In [ ]:
# Toma aproximadamente 20 a 30 minutos.
!./run.sh --stage 11 --stop_stage 11 --ngpu 1

### Etapa 12: Decodificacion:
En esta etapa evaluamos el modelo, generando texto de nuestras listas de evaluacion. 
Recuerda deshabilitar el uso del modelo de lenguaje, usando: `--use-lm false`

In [ ]:
# Aprox 10 mins.
!./run.sh --inference_nj 4 --stage 12 --stop_stage 12 --use_lm false

### Etapa 13: Puntuacion
Podemos encontrar el porcentaje de errores, por palabra (o WER), por caracter (CER), etc. para cada test

In [ ]:
!./run.sh --stage 13 --stop_stage 13 --use_lm false